In [1]:
# Feature Engineering for Water Pump Classification
import pandas as pd
import numpy as np
import os
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import cross_val_score
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline
from sklearn.model_selection import GridSearchCV
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, confusion_matrix
import joblib
# Load dataset (adjust path as needed)
data_path = '../Data/train.csv'
labels_path = '../Data/train_labels.csv' 

df = pd.read_csv(data_path)
train = pd.read_csv(data_path)
labels = pd.read_csv(labels_path)

import warnings
warnings.filterwarnings("ignore")

In [131]:
def pumpit_clean(df :pd.core.frame.DataFrame):
    # ``` This function requires: numpy, pandas, os,
    # ```
    pd.set_option('future.no_silent_downcasting', True)
    # handle extra imports here
    from sklearn.preprocessing import MinMaxScaler
    from sklearn.preprocessing import LabelEncoder

    ## Numerical Variables

    # amount_tsh: cap at 15000
    df['amount_tsh'] = df['amount_tsh'].apply(lambda x: min(x, 15000))
    # might want to replace this with bins
    scaler = MinMaxScaler()
    df['amount_tsh'] = scaler.fit_transform(df[['amount_tsh']]) # names kept consistent
    # gps_height
    # # Replace invalid gps_height values (e.g. 0 or negative)
    df['gps_height'] = df['gps_height'].apply(lambda x: np.nan if x <= 0 else x)
    # Fill using median per basin
    df['gps_height'] = df.groupby('basin')['gps_height'].transform(
        lambda x: x.fillna(x.median())
    )
    # Fill any still missing using region median
    df['gps_height'] = df.groupby('region')['gps_height'].transform(
        lambda x: x.fillna(x.median())
    )
    df['gps_height'] = scaler.fit_transform(df[['gps_height']])

    # location:
    df['longitude'] = df['longitude'].replace(0, np.nan)
    df['latitude'] = df['latitude'].where(df['latitude'] < -0.5, np.nan) # too close to the equator
    for i in ['latitude','longitude']:
        df[i] = df.groupby('lga')[i].transform(lambda x: x.fillna(x.median))
        df[i] = df.groupby('region')[i].transform(lambda x: x.fillna(x.median))

    # Fill population using median by district_code
    df['population'] = df.groupby('lga')['population'].transform(
        lambda x: x.fillna(x.median())
    )
    # Fill any still missing with median by region, then overall median
    df['population'] = df.groupby('region')['population'].transform(
        lambda x: x.fillna(x.median())
    )
    df['population'] = df['population'].fillna(df.population.median)
    # Bin the outcome, see how it behaves
    df['population'] = pd.cut(df['population'], [-1,1,25,90,160,260,9999999], labels=[0,0.1,0.2,0.4,0.6,0.9])
    df['population'] = df['population'].astype(float)
    # Construction year ??
    df['construction_year'] = df['construction_year'].replace(0, np.nan)
    df['date_recorded'] = pd.to_datetime(df['date_recorded'])
    df['recorded_year'] = df['date_recorded'].dt.year
    #Impute using region + installer
    df['construction_year'] = df.groupby(['region', 'installer'])['construction_year'].transform(
        lambda x: x.fillna(x.median())
    )
    #Impute using region only (for rows still missing)
    df['construction_year'] = df.groupby('region')['construction_year'].transform(
        lambda x: x.fillna(x.median())
    )
    #Use recorded year - 5
    df['construction_year'] = df['construction_year'].fillna(df['recorded_year'] - 5)
    df['construction_year'] = scaler.fit_transform(df[['construction_year']])

    ### Encode categorical variables

    # Encode 'quantity' (and typo fix: 'insufficent' -> 'insufficient')
    df['quantity'] = df['quantity'].replace({
        'enough': 1,
        'seasonal': 0.6,
        'insufficient': 0.4,
        'dry': 0,
        'unknown': 0
    })
    df['quantity'] = df['quantity'].astype(float)
    # Encode 'water_quality' as binary: good = 1, else 0
    df['water_quality'] = np.where(df['water_quality'] == 'soft', 1, 0)
    # Encode 'waterpoint_type' (1 = preferred type, 0 = everything else)
    preferred_waterpoint = ['hand pump', 'communal standpipe']
    df['waterpoint_type'] = df['waterpoint_type'].apply(lambda x: 1 if x in preferred_waterpoint else 0)
    # Encode 'payment' as binary: never pay = 0, else = 1
    df['payment'] = np.where(df['payment'] == 'never pay', 0, 1)
    # Encode 'source' (1 = preferred sources, 0 = everything else)
    preferred_sources = ['spring', 'river', 'rainwater harvesting']
    df['source'] = df['source'].apply(lambda x: 1 if x in preferred_sources else 0)
    # Encode 'payment' as binary: never pay = 0, else = 1
    df['extraction_type_class'] = np.where(df['extraction_type_class'] == 'gravity', 0, 1)

    #  Drop other columns and only keep these:
    df = df[['amount_tsh',
             'gps_height',
 #            'longitude',
 #            'latitude',
             'population',
             'construction_year',
             'extraction_type_class',
             'payment',
            'water_quality',
            'quantity',
            'source',
            'waterpoint_type',
       ]]
    df['populationXquantity'] = df['population'] * df['quantity']
    df['waterpointXsource'] = df['waterpoint_type'] * df['source']
    df['quantityXsource'] = df['quantity'] * df['source']
    df['constr_yearXpopulation'] = df['construction_year'] * df['population']
    df['waterpointXconst_year'] = df['waterpoint_type'] * df['construction_year']
    return df


In [132]:
df = pumpit_clean(train)
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 59400 entries, 0 to 59399
Data columns (total 15 columns):
 #   Column                  Non-Null Count  Dtype  
---  ------                  --------------  -----  
 0   amount_tsh              59400 non-null  float64
 1   gps_height              59400 non-null  float64
 2   population              59400 non-null  float64
 3   construction_year       59400 non-null  float64
 4   extraction_type_class   59400 non-null  int64  
 5   payment                 59400 non-null  int64  
 6   water_quality           59400 non-null  int64  
 7   quantity                59400 non-null  float64
 8   source                  59400 non-null  int64  
 9   waterpoint_type         59400 non-null  int64  
 10  populationXquantity     59400 non-null  float64
 11  waterpointXsource       59400 non-null  int64  
 12  quantityXsource         59400 non-null  float64
 13  constr_yearXpopulation  59400 non-null  float64
 14  waterpointXconst_year   59400 non-null

In [133]:
# load labels
labels = pd.read_csv(os.path.join(labels_path))
# separate labels into two variables
y1 = labels.copy(deep=False)
y2 = labels.copy(deep=False)
y1.status_group = np.where(y1["status_group"] == "functional", 1, 0)
y2.status_group = np.where(y2["status_group"] == "non functional", 1, 0)
# make copy of training data
y1 = y1.drop(columns=['id'])
y2 = y2.drop(columns=['id'])
X = df
# split the data
X_train, X_test, y_train, y_test = train_test_split(
    X, y1, test_size=0.3, random_state=42
)


In [120]:
# pipeline
pipeline = Pipeline([
    ('scaler', StandardScaler()),
    ('logreg', LogisticRegression(max_iter=1000))
])

# Parameter grid
param_grid = {
    'logreg__C': [0.01, 0.1],
    'logreg__penalty': ['l2'],
    'logreg__solver': ['lbfgs', 'saga']
}

# Grid search
grid_search = GridSearchCV(pipeline, param_grid, cv=2, scoring='accuracy')
grid_search.fit(X_train, y_train)  # this is essential

# Check best_estimator_
print(" Type:", type(grid_search))
print(" Best Estimator:", grid_search.best_estimator_)

# Save to file
joblib.dump(grid_search.best_estimator_, 'best_logistic_model.pkl')


 Type: <class 'sklearn.model_selection._search.GridSearchCV'>
 Best Estimator: Pipeline(steps=[('scaler', StandardScaler()),
                ('logreg',
                 LogisticRegression(C=0.01, max_iter=1000, solver='saga'))])


['best_logistic_model.pkl']

In [122]:
# Evaluate on the test set

y_test_pred = grid_search.best_estimator_.predict(X_test)

print(" Classification Report on Test Set:")
print(classification_report(y_test, y_test_pred))

print(" Confusion Matrix:")
print(confusion_matrix(y_test, y_test_pred))

 Classification Report on Test Set:
              precision    recall  f1-score   support

           0       0.71      0.60      0.65      8096
           1       0.71      0.80      0.75      9724

    accuracy                           0.71     17820
   macro avg       0.71      0.70      0.70     17820
weighted avg       0.71      0.71      0.71     17820

 Confusion Matrix:
[[4881 3215]
 [1975 7749]]


In [128]:
X2 = df
X2.outcome1 = y1
X2.outcome1 = y_test_pred
X2 = X2[X2.outcome1 == 0].copy(deep=False)
X2.head()
# second round
X_train, X_test, y_train, y_test = train_test_split(
    X2, y2, test_size=0.3, random_state=42
)


ValueError: Item wrong length 17820 instead of 59400.

In [138]:
## In other news, here are the results with the added interaction variables
# Encode target labels
le = LabelEncoder()
y_encoded = le.fit_transform(labels['status_group']) 


# Define features and target
X = df
y = y_encoded
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, stratify=y, random_state=42
)


In [139]:
# Grid search
grid_search = GridSearchCV(pipeline, param_grid, cv=5, scoring='accuracy')
grid_search.fit(X_train, y_train)  # this is essential

# Check best_estimator_
print(" Type:", type(grid_search))
print(" Best Estimator:", grid_search.best_estimator_)

# Save to file
joblib.dump(grid_search.best_estimator_, 'best_logistic_model.pkl')

# Evaluate on the test set

y_test_pred = grid_search.best_estimator_.predict(X_test)

print(" Classification Report on Test Set:")
print(classification_report(y_test, y_test_pred, target_names=le.classes_))

print(" Confusion Matrix:")
print(confusion_matrix(y_test, y_test_pred))

 Type: <class 'sklearn.model_selection._search.GridSearchCV'>
 Best Estimator: Pipeline(steps=[('scaler', StandardScaler()),
                ('logreg',
                 LogisticRegression(C=0.1, max_iter=1000, solver='saga'))])
 Classification Report on Test Set:
                         precision    recall  f1-score   support

             functional       0.69      0.84      0.76      6452
functional needs repair       0.00      0.00      0.00       863
         non functional       0.69      0.60      0.64      4565

               accuracy                           0.69     11880
              macro avg       0.46      0.48      0.47     11880
           weighted avg       0.64      0.69      0.66     11880

 Confusion Matrix:
[[5442    0 1010]
 [ 634    0  229]
 [1822    0 2743]]
